In [1]:
import os
import gc
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50V2, ResNet50
from tensorflow.keras.layers.experimental.preprocessing import RandomCrop,CenterCrop, RandomRotation
from tensorflow.keras.applications import EfficientNetB0

import cv2

In [2]:
# load data from tensorflow framework
(x_train, x_test), (y_train, y_test) = np.load('./img_data3.npy',allow_pickle=True)

# Stack train data and test data to form single array 
az_data = np.vstack([x_train, x_test])
print(az_data.shape)
# Horizontal stacking labels of train and test set
az_labels = np.hstack([y_train, y_test])
print(az_labels.shape)

(187000, 28, 28)
(187000,)


In [3]:
# Uniques and counts of train labels
unique_train, counts_train = np.unique(y_train, return_counts= True)
print(f"Value counts of y_train modalities: {counts_train}\n")

# Uniques and counts of test labels
unique_test, counts_test = np.unique(y_test, return_counts= True)
print(f"Value counts of y_test modalities: {counts_test}")

Value counts of y_train modalities: [8277 8238 8229 8336 8269 8267 8149 8298 8239 8230 8313 8170 8208 8305
 8213 8288 8221]

Value counts of y_test modalities: [2723 2762 2771 2664 2731 2733 2851 2702 2761 2770 2687 2830 2792 2695
 2787 2712 2779]


In [4]:
az_labels

array([10,  3, 13, ...,  2,  7,  1])

In [5]:
z = np.hstack([az_labels])

In [6]:
z.shape

(187000,)

In [7]:
a= az_labels
a

array([10,  3, 13, ...,  2,  7,  1])

In [8]:

# stack the A-Z data and labels with the MNIST digits data and
# Each image in the A-Z and MNIST digts datasets are 28x28 pixels;
# However, the architecture we're using is designed for 32x32 images,
# So we need to resize them to 32x32

az_data = [cv2.resize(image, (32, 32)) for image in az_data]
az_data = np.array(az_data, dtype="float32")

# add a channel dimension to every image in the dataset and scale the
# pixel intensities of the images from [0, 255] down to [0, 1]

az_data = np.expand_dims(az_data, axis=-1)
az_data /= 255.0

In [9]:
az_data

array([[[[0.20784314],
         [0.03921569],
         [0.14117648],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.2       ],
         [0.03921569],
         [0.02745098],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.0627451 ],
         [0.01176471],
         [0.        ],
         ...,
         [0.09411765],
         [0.        ],
         [0.        ]],

        ...,

        [[0.20392157],
         [0.25882354],
         [0.7058824 ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.20784314],
         [0.21176471],
         [0.5686275 ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.20784314],
         [0.22352941],
         [0.1764706 ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]]],


       [[[0.2       ],
         [0.2       ],
         [0.06

In [10]:
az_data.shape

(187000, 32, 32, 1)

In [11]:
le = LabelBinarizer()
az_labels = le.fit_transform(az_labels)
print(az_labels.shape)

counts = az_labels.sum(axis=0)

# account for skew in the labeled data
classTotals = az_labels.sum(axis=0)
classWeight = {}

# loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]

(187000, 17)


In [12]:
classWeight

{0: 1.0,
 1: 1.0,
 2: 1.0,
 3: 1.0,
 4: 1.0,
 5: 1.0,
 6: 1.0,
 7: 1.0,
 8: 1.0,
 9: 1.0,
 10: 1.0,
 11: 1.0,
 12: 1.0,
 13: 1.0,
 14: 1.0,
 15: 1.0,
 16: 1.0}

In [13]:
(trainX, testX, trainY, testY) = train_test_split(az_data,az_labels, test_size=0.20, stratify=az_labels, random_state=42)

In [14]:
trainX.shape

(149600, 32, 32, 1)

In [15]:
from keras.preprocessing.image import ImageDataGenerator
from skimage import io
aug = ImageDataGenerator(
rotation_range=10,
zoom_range=0.05,
width_shift_range=0.1,
height_shift_range=0.1,
shear_range=0.15,
horizontal_flip=False,
fill_mode="nearest")


In [16]:
batch_size = 32
epochs = 50

In [17]:
enet = ResNet50(
        input_shape=(32, 32, 1),
        weights=None,
        include_top=False
    )

model = tf.keras.Sequential([
        enet,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(17, activation='softmax')
    ])


callbacks = [ModelCheckpoint(filepath='best_model_index.h5', save_weights_only = True,
                             monitor='val_accuracy' ,mode='max')]
early = EarlyStopping(monitor="val_accuracy", mode="max", patience=20, verbose = 1)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 1, 1, 2048)        23581440  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 17)                34833     
Total params: 23,616,273
Trainable params: 23,563,153
Non-trainable params: 53,120
_________________________________________________________________


In [18]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("using {} device".format(device))

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

C:\ProgramData\Anaconda3\envs\idip_2021\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using cuda device
Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [19]:
history = model.fit(
    aug.flow(trainX, trainY, batch_size=batch_size),
    epochs = epochs, 
    validation_data=(testX, testY),
    class_weight=classWeight,
    verbose=1,
    callbacks=[callbacks, early]).to(device)

Epoch 1/50
4675/4675 [==============================] - 758s 159ms/step - loss: 0.3571 - accuracy: 0.9242 - val_loss: 0.0195 - val_accuracy: 0.9937
Epoch 2/50
4675/4675 [==============================] - 597s 128ms/step - loss: 0.1506 - accuracy: 0.9652 - val_loss: 3.3174 - val_accuracy: 0.9078
Epoch 3/50
4675/4675 [==============================] - 530s 113ms/step - loss: 0.0846 - accuracy: 0.9834 - val_loss: 0.0120 - val_accuracy: 0.9966
Epoch 4/50
4675/4675 [==============================] - 531s 114ms/step - loss: 0.0614 - accuracy: 0.9876 - val_loss: 0.0022 - val_accuracy: 0.9990
Epoch 5/50
4675/4675 [==============================] - 531s 114ms/step - loss: 0.0325 - accuracy: 0.9921 - val_loss: 0.0018 - val_accuracy: 0.9994
Epoch 6/50
4675/4675 [==============================] - 532s 114ms/step - loss: 0.0229 - accuracy: 0.9940 - val_loss: 9.0335e-04 - val_accuracy: 0.9998
Epoch 7/50
4675/4675 [==============================] - 533s 114ms/step - loss: 0.0180 - accuracy: 0.9954 - 

AttributeError: 'History' object has no attribute 'to'

In [21]:
model.save('ResNet_index.h5')

In [74]:
model_ex = load_model('ResNet_index.h5')

In [77]:
yhat = model_ex.predict(trainX)

In [82]:
print(np.argmax(yhat[2]))

13


In [83]:
print(trainY[2])

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
